In [ ]:
#import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rnd

from sklearn.model_selection import train_test_split

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

data_df = pd.read_csv('../input/student-por.csv')
train_df, test_df = train_test_split(data_df, test_size=0.3)
combine = [train_df, test_df]

train_df = train_df.drop(['famsize', 'school'], axis=1)
test_df = test_df.drop(['famsize','school'], axis=1)
combine = [train_df,test_df]

#Sets sex from object to integer, female = 1, male = 0
for dataset in combine:
    dataset['sex'] = dataset['sex'].map( {'F': 1, 'M': 0} ).astype(int)

#Parents r together = 1, else 0
for dataset in combine:
    dataset['Pstatus'] = dataset['Pstatus'].map({'T': 1, 'A':0}).astype(int)

#Map Mjob & Fjob into mEmployed, fEmployed 1 for employed 0 for unemployed
for dataset in combine:
    dataset['mEmployed'] = 1
    dataset['fEmployed'] = 1
    dataset.loc[(dataset['Mjob'] == 'at_home'), 'mEmployed'] = 0
    dataset.loc[(dataset['Fjob'] == 'at_home'), 'fEmployed'] = 0
#Combine Medu and mEmployed into Mstatus
for dataset in combine:
    dataset['mStatus'] = dataset.mEmployed + 0*dataset.Medu
    dataset['fStatus'] = dataset.fEmployed + 0*dataset.Fedu

#remove mother/father job/education as it is being measured in mStatus/fStatus
train_df = train_df.drop(['Mjob', 'Fjob', 'Medu', 'Fedu', 'mEmployed', 'fEmployed'], axis=1)
test_df = test_df.drop(['Mjob', 'Fjob', 'Medu', 'Fedu', 'mEmployed', 'fEmployed'], axis =1)
#remove reason & guardian
train_df = train_df.drop(['reason', 'guardian', 'address'], axis=1)
test_df = test_df.drop(['reason', 'guardian', 'address'], axis=1)
#remove travel time
train_df = train_df.drop(['traveltime', 'schoolsup', 'paid', 'nursery', 'internet', 'health'], axis=1)
test_df = test_df.drop(['traveltime', 'schoolsup', 'paid', 'nursery', 'internet', 'health'], axis=1)
combine = [train_df, test_df]

#Combine G# into grades
for dataset in combine:
    dataset['grades'] = (dataset.G1 + dataset.G2 + dataset.G3)//3

#drop G1,G2,G3 as it is now being accumulated in grades
train_df = train_df.drop(['G1','G2','G3'], axis=1)
test_df = test_df.drop(['G1','G2','G3'], axis=1)
combine = [train_df, test_df]
for dataset in combine:
    dataset['famsup'] = dataset['famsup'].map({'yes': 1, 'no': 0}).astype(int)

#combine family info into family
for dataset in combine:
    dataset['family'] = dataset['mStatus'] + dataset['fStatus'] + 0*dataset['famsup'] + 0*dataset['Pstatus'] + dataset['famrel']

#dropped since it is being evaluated in family
#train_df = train_df.drop(['mStatus','fStatus','famsup', 'Pstatus', 'famrel'], axis=1)
#test_df = test_df.drop(['mStatus','fStatus','famsup', 'Pstatus', 'famrel'], axis=1)
train_df = train_df.drop(['mStatus','fStatus','famsup', 'famrel'], axis=1)
test_df = test_df.drop(['mStatus','fStatus','famsup', 'famrel'], axis=1)
combine = [train_df, test_df]

#quality of student
for dataset in combine:
    dataset['activities'] = dataset['activities'].map({'yes': 1, 'no': 0}).astype(int)
    dataset['higher'] = dataset['higher'].map({'yes': 1,'no':0}).astype(int)
    dataset['qStudent'] = dataset.activities + dataset.higher + dataset.studytime - 0*dataset.failures
train_df = train_df.drop(['activities','higher','studytime', 'failures'], axis=1)
test_df = test_df.drop(['activities','higher','studytime', 'failures'], axis=1)
combine = [train_df, test_df]
#social of student
for dataset in combine:
    dataset['romantic'] = dataset['romantic'].map({'yes':1, 'no':0}).astype(int)
    dataset['sStudent'] = dataset.romantic + 0*dataset.freetime + dataset.goout
train_df = train_df.drop(['romantic','goout','freetime'], axis=1)
test_df = test_df.drop(['romantic','goout','freetime'], axis=1)
combine = [train_df, test_df]

#set Dalc and Walc into alc
for dataset in combine:
    dataset['alc'] = dataset.Dalc + dataset.Walc
train_df = train_df.drop(['Dalc','Walc'], axis=1)
test_df = test_df.drop(['Dalc','Walc'], axis=1)
combine = [train_df, test_df]

mean = train_df['alc'].mean()*0.7 + test_df['alc'].mean()*0.3
#mean = 4
for dataset in combine:
    dataset.loc[dataset['alc'] <= mean, 'alc'] = 0
    dataset.loc[dataset['alc'] > mean, 'alc'] = 1

#new stuff?
#mean = train_df['absences'].mean()*0.7 + test_df['absences'].mean()*0.3
#print(mean)
#mean = 15
for dataset in combine:
    dataset.loc[dataset['absences'] <= 5, 'absences'] = 0
    dataset.loc[dataset['absences'] <= 10, 'absences'] = 1
    dataset.loc[dataset['absences'] <= 25, 'absences'] = 2
    dataset.loc[dataset['absences'] > 25, 'absences'] = 3
    
#mean = train_df['grades'].mean()*0.7 + test_df['grades'].mean()*0.3
#mean = mean/4
for dataset in combine:
    dataset.loc[dataset['grades'] <= 20, 'grades'] = 1
    dataset.loc[dataset['grades'] <= 30, 'grades'] = 2
    dataset.loc[dataset['grades'] <= 45, 'grades'] = 3
    dataset.loc[dataset['grades'] < 45, 'grades'] = 4
    
for dataset in combine:
    dataset.loc[dataset['age'] < 18, 'age'] = 0
    dataset.loc[dataset['age'] >= 18, 'age'] =1
    
#end of new stuff

train_df = train_df[['alc','sex','age','absences','grades','family','qStudent','sStudent', 'Pstatus']]
test_df = test_df[['alc','sex','age','absences','grades','family','qStudent','sStudent', 'Pstatus']]

#train_df = train_df.drop(['absences'], axis=1)
#test_df = test_df.drop(['absences'], axis=1)

#split data
X_train = train_df.drop("alc", axis=1)
Y_train = train_df["alc"]
X_test  = test_df.drop("alc", axis=1)
Y_test  = test_df["alc"]


# Logistic Regression
print('Logistic Regression')
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
lr_pred = logreg.predict(X_test)
score_log = logreg.score(X_train, Y_train)
acc_log =accuracy_score(Y_test,lr_pred)
print(classification_report(Y_test,lr_pred))
#Correlation
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

print(coeff_df.sort_values(by='Correlation', ascending=False))
#multi layer perceptron ANN
print('')
print('Multilayer Perceptron')
mlp = MLPClassifier(hidden_layer_sizes=(8,8,8), max_iter=5000)
mlp.fit(X_train,Y_train)
mlp_predictions = mlp.predict(X_test)

score_mlp = mlp.score(X_train, Y_train)
acc_mlp = accuracy_score(Y_test,mlp_predictions)

print(classification_report(Y_test,mlp_predictions))


# Decision Tree
print('Decision Tree')
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
dt_pred = decision_tree.predict(X_test)
score_dt = decision_tree.score(X_train, Y_train)
acc_dt = accuracy_score(Y_test,dt_pred)

print(classification_report(Y_test,dt_pred))

models = pd.DataFrame({
    'Model': ['Logistic Regression', 'Multilayer Perceptron', 'Decision Tree'],
    'Score': [score_log, score_mlp, score_dt],
    'Accuracy': [acc_log, acc_mlp, acc_dt]})
print(models.sort_values(by='Score', ascending=False))
print('')
print(train_df.columns.values)